In [1]:
# if german spacy not present uncomment this:
#!python3 -m spacy download de

In [2]:
import torch
import numpy  as np
from torch import nn, optim
import time
import torch.nn.functional as F

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import spacy
spacy_english = spacy.load('en')
spacy_german = spacy.load('de')

In [3]:
# tokenize languages
def tokenize_german(text):
    return [token.text for token in spacy_german.tokenizer(text)]

# reversing order has been shown to imporve model!!! 
def tokenize_english(text):
    return [token.text for token in spacy_english.tokenizer(text)][::-1] 



In [4]:
SOURCE = Field(tokenize = tokenize_english, 
               init_token = "<sos>",
               eos_token = "<eso>",
               lower = True
              )

TARGET = Field(tokenize = tokenize_german, 
               init_token = "<sos>",
               eos_token = "<eso>",
               lower = True
              )

train_data, valid_data, test_data = Multi30k.splits(exts = ('.en', '.de'),
                                                    fields= (SOURCE, TARGET))


/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [5]:
print(train_data.examples[0].src)
print(train_data.examples[0].trg)

['.', 'bushes', 'many', 'near', 'outside', 'are', 'males', 'white', ',', 'young', 'two']
['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.']


In [6]:
len(train_data.examples), len(test_data.examples), len(test_data.examples)

(29000, 1000, 1000)

In [7]:
SOURCE.build_vocab(train_data, min_freq = 2)
TARGET.build_vocab(train_data, min_freq = 2)
len(SOURCE.vocab), len(TARGET.vocab)

(5893, 7854)

In [8]:
device =  torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [9]:
batch_size = 32
train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data),
                                                                      batch_size = batch_size, 
                                                                      device = device)

/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [10]:
# building the encoder

In [31]:
class Encoder(nn.Module):
    def __init__(self, input_dims, emd_dims, hidden_dims, n_layers, dropout):
        super().__init__()
        self.hid_dims = hidden_dims
        self.emd_dims = emd_dims
        self.n_layers = n_layers
        self.dropout = dropout
        self.input_dims = input_dims
        
        self.embeddings = nn.Embedding(input_dims, emd_dims)
        self.rnn = nn.LSTM(emd_dims, hidden_dims, n_layers, dropout = dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        embedded = self.dropout(self.embeddings(src))
        outputs, (h, cell) = self.rnn(embedded)
        return h, cell

        
class Decoder(nn.Module):
    def __init__(self, output_dims, emd_dims, hidden_dims, n_layers, dropout):
        super().__init__()
        self.hid_dims = hidden_dims
        self.emd_dims = emd_dims
        self.n_layers = n_layers
        self.dropout = dropout
        self.output_dims = output_dims
        
        self.embeddings = nn.Embedding(output_dims, emd_dims)
        self.rnn = nn.LSTM(emd_dims, hidden_dims, n_layers, dropout = dropout)
        self.fc_out = nn.Linear(hidden_dims, output_dims)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input_data, h, cell):
        input_data = input_data.unsqueeze(0)
        embedded = self.dropout(self.embeddings(input_data))
        outputs, (h, cell) = self.rnn(embedded)
        pred = self.fc_out(outputs.squeeze(0))
        return pred, h, cell


In [38]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_rate=0.5):
        batch_size = trg.shape[1]
        target_length = trg.shape[0]
        target_vocab_size = self.decoder.output_dims
        outputs = torch.zeros(target_length, batch_size, target_vocab_size).to(self.device)
        
        h, cell = self.encoder(src)
        input_tok = trg[0,:]
        for t  in range(1, target_length):
            output, h, cell = self.decoder(input_tok, h, cell)
            outputs[t] = output 
            top = output.argmax(1)
            input_tok = trg[t] if (np.random.random() < teacher_forcing_rate)  else top
        return outputs
        
        

In [39]:
# make an instance ready to be trained:

input_dimension = len(SOURCE.vocab)
output_dimension = len(TARGET.vocab)
encoder_embedding_dimension = 256
decoder_embedding_dimension = 256
hidden_layer_dimension = 512
number_of_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5

encod = Encoder(input_dimension, 
                encoder_embedding_dimension, 
                hidden_layer_dimension, 
                number_of_layers, 
                encoder_dropout)

decod = Decoder(output_dimension, 
                decoder_embedding_dimension, 
                hidden_layer_dimension, 
                number_of_layers, 
                decoder_dropout)

model = Seq2Seq(encod, decod, device).to(device)


In [40]:
# training loop
def initialize_weights(m):
    for name, param, in m.named_parameters():
        nn.init.uniform_(param.data, -0.1, 0.1)

model.apply(initialize_weights)

Seq2Seq(
  (encoder): Encoder(
    (embeddings): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embeddings): Embedding(7854, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=7854, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [41]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = TARGET.vocab.stoi[TARGET.pad_token])


In [46]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        optimizer.zero_grad()
        output = model(src, trg)
    
        output_dims = output.shape[-1]
        output = output[1:].view(-1, output_dims)
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss/len(iterator)

def evaluate(model, iterator, optimizer, criterion):
    
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg
            # turn off teacher forcing
            output = model(src, trg, 0)

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)
        
    
        

In [64]:
epochs = 10
grad_clip = 1
lowest_validation_loss = float('inf')

for epoch in range(epochs):
    start_time = time.time()
    
    training_loss = train(model, train_iterator, optimizer, criterion, grad_clip)
    validation_loss = evaluate(model, valid_iterator, optimizer, criterion)
    end_time = time.time()
    
    if validation_loss <  lowest_validation_loss:
        lowest_validation_loss = validation_loss
        torch.save(model.state_dict(), 'seq2seq.pt')
    
    print(f"Epoch {epoch}{epochs}, time : {np.round(end_time-start_time)}s")
    print(f"Train loss: {training_loss}")
    print(f"VAlid loss: {validation_loss}")
    


Epoch 010, time : 1697.0s
Train loss: 4.483527246094591
VAlid loss: 5.154193684458733
Epoch 110, time : 2848.0s
Train loss: 4.452940569165786
VAlid loss: 5.136019766330719
Epoch 210, time : 1598.0s
Train loss: 4.433290380890077
VAlid loss: 5.144728496670723
Epoch 310, time : 2099.0s
Train loss: 4.426760333375605
VAlid loss: 5.132786333560944
Epoch 410, time : 1903.0s
Train loss: 4.395755526249306
VAlid loss: 5.141524598002434
Epoch 510, time : 2768.0s
Train loss: 4.393345655916544
VAlid loss: 5.1446192264556885
Epoch 610, time : 1584.0s
Train loss: 4.385759874636178
VAlid loss: 5.2161654978990555
Epoch 710, time : 1626.0s
Train loss: 4.375559087099288
VAlid loss: 5.157773554325104
Epoch 810, time : 2931.0s
Train loss: 4.362987329836323
VAlid loss: 5.150640517473221
Epoch 910, time : 2405.0s
Train loss: 4.365790959624235
VAlid loss: 5.1443169713020325


In [65]:
model.load_state_dict(torch.load('seq2seq.pt'))

test_loss = evaluate(model, test_iterator, optimizer, criterion)

print(f'Test Loss: {test_loss:.4f}')

Test Loss: 5.0763


In [66]:
def translate(model, iterator):
    
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg
            # turn off teacher forcing
            output = model(src, trg, 0)
            
            preds = torch.tensor([[torch.argmax(x).item()] for x in output])
            
            input_text = [SOURCE.vocab.itos[x] for x in  src][1:-1][::-1] # drop start & end
            output_text = [TARGET.vocab.itos[x] for x in  trg][1:-1] # drop start & end
            predict_text = [TARGET.vocab.itos[x] for x in  preds][1:-1] # drop start & end
            
            print(input_text)
            print(output_text)
            print("prediction")
            print(predict_text)
        
    return None
    

In [67]:
_, _, eval_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = 1, 
    device = device)

In [68]:
output = translate(model, eval_iterator)

['two', 'men', 'wearing', 'hats', '.']
['zwei', 'männer', 'mit', 'mützen', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>']
['young', 'woman', 'climbing', 'rock', 'face']
['junge', 'frau', 'klettert', 'auf', 'felswand']
prediction
['ein', 'mann', 'in', 'einem', '<unk>']
['a', 'woman', 'is', 'playing', 'volleyball', '.']
['eine', 'frau', 'spielt', 'volleyball', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>']
['three', 'men', 'are', 'walking', 'up', 'hill', '.']
['drei', 'männer', 'gehen', 'bergauf', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>']
['an', 'army', 'officer', 'is', 'inspecting', 'something', '.']
['ein', '<unk>', 'inspiziert', 'etwas', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>']
['a', 'rock', 'concert', 'is', 'taking', 'place', '.']
['ein', 'rockkonzert', 'findet', 'statt', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>']
['people', 'sit', 'inside', 'a', 'train', '.']
['leute', 'sitzen', 'in', 'einem', 'zug', '.']
prediction
['ein',

['crowds', 'of', 'people', 'are', 'all', 'riding', 'bicycles', '.']
['<unk>', ',', 'die', 'alle', 'fahrrad', 'fahren', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>']
['two', 'children', 'are', 'playing', 'on', 'a', 'bicycle', '.']
['zwei', 'kinder', 'spielen', 'auf', 'einem', 'fahrrad', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>']
['three', 'men', 'are', 'cooking', 'in', 'a', 'kitchen', '.']
['drei', 'männer', 'kochen', 'in', 'einer', 'küche', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>']
['people', 'playing', 'a', 'game', 'in', 'the', 'pool', '.']
['leute', 'spielen', 'ein', 'spiel', 'im', 'pool', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>']
['two', 'cars', 'are', 'driving', 'on', 'a', 'racetrack', '.']
['zwei', 'autos', 'fahren', 'auf', 'einer', 'rennstrecke', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>']
['construction', 'workers', 'having', 'a', 'discussion', 'by

['a', 'man', 'cooking', 'food', 'on', 'the', 'stove', '.']
['ein', 'mann', 'bereitet', 'am', 'herd', 'essen', 'zu', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>']
['three', 'teenagers', 'in', 'a', 'subway', 'playing', 'around', '.']
['drei', 'teenager', 'in', 'einer', 'u-bahn', 'albern', 'herum', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>']
['the', 'boy', 'is', 'wakeboarding', 'on', 'the', 'lake', '.']
['ein', 'junge', 'beim', 'wakeboarden', 'auf', 'dem', 'see', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>']
['a', 'blond', 'child', 'swinging', 'on', 'a', 'swing', '.']
['ein', 'blondes', 'kind', 'schaukelt', 'auf', 'einer', 'schaukel', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>']
['an', 'asian', 'man', 'is', 'cooking', 'food', 'outdoors', '.']
['ein', 'asiate', 'bereitet', 'im', 'freien', 'essen', 'zu', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>'

['a', 'brown', 'dog', 'runs', 'down', 'the', 'sandy', 'beach', '.']
['ein', 'brauner', 'hund', 'läuft', 'über', 'den', 'sandstrand', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>']
['girl', 'wearing', 'radio', 't', '-', 'shirt', 'has', 'open', 'mouth']
['ein', 'mädchen', 'in', 'einem', '<unk>', 'mit', 'offenem', 'mund']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>']
['the', 'two', 'kids', 'are', 'playing', 'at', 'the', 'playground', '.']
['die', 'zwei', 'kinder', 'spielen', 'auf', 'dem', 'spielplatz', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>']
['a', 'dog', 'is', 'jumping', 'through', 'a', '<unk>', 'obstacle', '.']
['ein', 'hund', 'springt', 'durch', 'ein', 'brennendes', 'hindernis', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>']
['a', 'boy', 'riding', 'a', 'skateboard', 'on', 'a', 'skateboarding', 'ramp']
['ein', 'junge', 'fährt', 'skateboard', 'auf', 'einer', '

['a', 'blond', 'girl', 'sleeps', 'on', 'a', 'brown', 'couch', '.']
['ein', 'blondes', 'mädchen', 'schläft', 'auf', 'einer', 'braunen', 'couch', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>', '<eso>']
['four', 'children', 'practice', 'karate', 'while', 'two', 'adults', 'watch', '.']
['vier', 'kinder', 'üben', 'karate', 'während', 'zwei', 'erwachsene', 'zusehen', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>', '<eso>']
['a', 'technician', 'preparing', 'a', 'sample', 'in', 'the', 'lab', '.']
['ein', '<unk>', 'bereitet', 'im', 'labor', 'eine', 'probe', 'vor', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>', '<eso>']
['a', 'man', 'is', 'playing', 'an', '<unk>', 'in', 'a', 'concert']
['ein', 'mann', 'spielt', 'ein', '<unk>', 'in', 'einem', 'konzert', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>', '<eso>']
['a', 'man', 'is', 'talking', 'on', 'a', 'cellphone', 'outside',

...

['a', 'man', 'walks', 'down', 'a', 'cobblestone', 'street', 'next', 'to', 'buildings', 'painted', 'tan', 'with', 'red', '<unk>', '.']
['ein', 'mann', 'geht', 'auf', 'einer', '<unk>', 'neben', 'gebäuden', 'mit', 'roten', '<unk>', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>']
['a', 'man', 'with', 'a', 'bushy', 'beard', 'and', 'a', 'baseball', 'cap', 'sits', 'on', 'a', 'park', 'bench', '.']
['ein', 'mann', 'mit', '<unk>', 'bart', 'und', 'baseballmütze', 'sitzt', 'auf', 'einer', 'parkbank', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>']
['fall', 'shoppers', 'and', 'bistro', 'food', 'lovers', 'caught', 'in', 'the', '<unk>', 'and', 'flow', 'of', 'the', 'city', '.']
['<unk>', 'und', '<unk>', 'lassen', 'sich', 'in', 'den', '<unk>', 'der', 'stadt', 'treiben', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>', '<eso>', '<eso>', '<eso

['an', 'elderly', 'man', 'in', 'a', 'light', 'blue', 'coat', 'works', 'on', 'a', 'car', 'on', 'the', 'roadside', '.']
['ein', 'älterer', 'mann', 'in', 'einem', 'hellblauen', 'mantel', 'arbeitet', 'am', 'straßenrand', 'an', 'einem', 'auto', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>']
['a', 'colorfully', 'dressed', 'young', 'man', 'with', 'visible', 'skin', '<unk>', 'is', 'sitting', 'and', 'smoking', 'a', 'cigarette', '.']
['ein', 'bunt', 'gekleideter', 'junger', 'mann', 'mit', 'sichtbaren', '<unk>', 'sitzt', 'und', 'raucht', 'eine', 'zigarette', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>']
['a', 'rocker', 'with', 'his', 'shirt', 'off', 'sings', 'into', 'a', 'microphone', 'while', 'he', 'plays', 'the', 'drums', '.']
['ein', 'rocker', 'mit', 'freiem', 'oberkörper', 'singt', 'in', 'ein', 'mikrofon', 'während', 'er',

['a', 'man', 'with', 'sunglasses', 'is', 'operating', 'a', 'construction', 'automobile', 'and', 'releasing', 'gravel', 'on', 'to', 'the', 'ground', '.']
['ein', 'mann', 'mit', 'sonnenbrille', 'bedient', 'ein', '<unk>', 'und', 'lädt', 'kies', 'auf', 'den', 'boden', 'ab', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>']
['an', 'older', 'man', 'with', 'a', 'cigarette', 'in', 'his', 'mouth', 'and', 'a', 'ball', 'cap', 'inspects', 'his', 'camera', '.']
['ein', 'älterer', 'mann', 'mit', 'einer', 'zigarette', 'im', 'mund', 'und', 'einer', 'mütze', 'inspiziert', 'seine', 'kamera', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>']
['a', 'woman', 'in', 'a', 'red', 'skirt', 'is', 'walking', 'on', 'the', 'street', 'with', 'graffiti', 'in', 'the', 'background', '.']
['eine', 'frau', 'in', 'einem', 'roten', 'rock', 'g

['a', 'man', 'on', 'a', 'scaffold', 'in', 'front', 'of', 'a', 'house', 'is', 'smiling', 'and', 'posing', 'for', 'the', 'photographer', '.']
['ein', 'mann', 'auf', 'einem', 'gerüst', 'vor', 'einem', 'haus', 'lächelt', 'und', 'posiert', 'für', 'den', 'fotografen', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>']
['two', 'female', 'members', 'of', 'team', 'usa', 'performing', 'a', 'jump', 'high', '-', 'five', 'surrounded', 'by', 'two', 'other', 'female', 'members', '.']
['zwei', '<unk>', 'der', '<unk>', 'klatschen', 'im', 'sprung', 'ab', ',', 'umgeben', 'von', 'zwei', 'weiteren', '<unk>', '.']
prediction
['ein', 'mann', 'in', 'einem', '<unk>', '.', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>', '<eso>']


KeyboardInterrupt: 